In [42]:
# list of supported layers
from tensorflow.keras.layers import (
    Input,
    Activation,
    AveragePooling2D,
    BatchNormalization,
    Conv2D,
    Dense,
    Dropout,
    Flatten,
    GlobalAveragePooling2D,
    GlobalMaxPooling2D,
    MaxPooling2D,
    ReLU,
    Softmax,
    )
from tensorflow.keras import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [43]:
# load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [44]:
# convert y_train and y_test to one-hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [64]:
print(y_test[4])

[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


In [65]:
import json
# reshape X_train and X_test to 4D tensor
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

#normalizing
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')




input_data = {
    "in":X_test[4].tolist()
}

with open("input_2.json", "w") as json_file:
    json.dump(input_data, json_file)


X_train /= 255.0
X_test /= 255.0


In [66]:
inputs = Input(shape=(28,28,1))
out = Conv2D(4, 3, use_bias=False)(inputs)
out = BatchNormalization()(out)
out = Activation('relu')(out)
out = MaxPooling2D()(out)
out = Conv2D(8, 3, use_bias=True, strides=2)(out)
out = ReLU()(out)
out = AveragePooling2D(pool_size=(2, 2))(out)
out = Flatten()(out)
# out = Dropout(0.5)(out)
out = Dense(10, activation="softmax")(out)
model = Model(inputs, out)

In [67]:
model.summary()

Model: "model_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_27 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 26, 26, 4)         36        
                                                                 
 batch_normalization_2 (Batc  (None, 26, 26, 4)        16        
 hNormalization)                                                 
                                                                 
 activation_2 (Activation)   (None, 26, 26, 4)         0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 4)        0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 6, 6, 8)           296

In [68]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['acc']
    )

In [69]:
model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/15
1875/1875 [==============================] - 11s 6ms/step - loss: 0.8908 - acc: 0.7205 - val_loss: 0.4889 - val_acc: 0.8570
Epoch 2/15
1875/1875 [==============================] - 10s 5ms/step - loss: 0.4473 - acc: 0.8647 - val_loss: 0.3822 - val_acc: 0.8897
Epoch 3/15
1875/1875 [==============================] - 10s 5ms/step - loss: 0.3595 - acc: 0.8922 - val_loss: 0.3115 - val_acc: 0.9093
Epoch 4/15
1875/1875 [==============================] - 10s 5ms/step - loss: 0.3012 - acc: 0.9096 - val_loss: 0.2683 - val_acc: 0.9207
Epoch 5/15
1875/1875 [==============================] - 10s 6ms/step - loss: 0.2645 - acc: 0.9200 - val_loss: 0.2345 - val_acc: 0.9266
Epoch 6/15
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2429 - acc: 0.9265 - val_loss: 0.2156 - val_acc: 0.9333
Epoch 7/15
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2276 - acc: 0.9309 - val_loss: 0.2030 - val_acc: 0.9374
Epoch 8/15
1875/1875 [==============================] -

In [70]:
model.save('model.h5')

In [71]:
from tensorflow.keras.models import Model

# Assume 'model' is your original model
# Extract the first 3 layers
new_model = Model(inputs=model.input, outputs=model.layers[2].output)

# Save the new model with the first 3 layers
new_model.save('model_first_3_layers.h5')

In [72]:
def convert_to_int(nested_list):
    # Check if the element is a list and recurse
    if isinstance(nested_list, list):
        return [convert_to_int(item) for item in nested_list]
    # If the element is a float, convert it to an integer
    elif isinstance(nested_list, float):
        return int(nested_list)
    else:
        return nested_list

In [ ]:
with open('../output/circuit_first.circom/output/output.json', 'r') as f:
    input_data = json.load(f)


# Flatten the input data if necessary, depending on your input model structure
# Assuming the input should be converted into a numpy array of floats
# You might need to adjust this depending on the exact shape of your input
input_array = np.array(input_data['conv2d_out'], dtype=np.float32)

# If the input shape doesn't match what your model expects, you may need to reshape it
# Assuming the model expects a batch size of 1 and a shape (height, width, channels) as the input shape
input_array = np.reshape(input_array, (1, *input_array.shape))  # Add a batch dimension if necessary



# Assume 'model' is your original model
# Get the outputs from layers 3 to 6
layer_outputs = [layer.output for layer in model.layers[3:7]]

# Create a new model that takes the original input and outputs the layers from 3 to 6
new_model = Model(inputs=model.layers[2].output, outputs=layer_outputs)

# Now, you can use `new_model` to predict and get outputs from layers 3 to 6
output = new_model.predict(input_array)
print(output[2].tolist())
output_test = convert_to_int(output[2].tolist())

formatted_output = {"in": output_test}

# Save the output as a JSON file
output_file = "../output/circuit_last.circom/input.json"
with open(output_file, "w") as f:
    json.dump(formatted_output, f, indent=4)
new_model.save("model_middle_layers.h5")
new_model.summary()


1/1 [==============================] - 0s 36ms/step
[[[[0.010385570116341114, 0.18124334514141083, 0.30283480882644653, -0.013509713113307953, 0.039314985275268555, 0.17047937214374542, 0.17117108404636383, -0.2989187240600586], [0.010385570116341114, 0.18124334514141083, 0.30283480882644653, -0.013509713113307953, 0.039314985275268555, 0.17047937214374542, 0.17117108404636383, -0.2989187240600586], [0.010385570116341114, 0.18124334514141083, 0.30283480882644653, -0.013509713113307953, 0.039314985275268555, 0.17047937214374542, 0.17117108404636383, -0.2989187240600586], [0.010385570116341114, 0.18124334514141083, 0.30283480882644653, -0.013509713113307953, 0.039314985275268555, 0.17047937214374542, 0.17117108404636383, -0.2989187240600586], [0.010385570116341114, 0.18124334514141083, 0.30283480882644653, -0.013509713113307953, 0.039314985275268555, 0.17047937214374542, 0.17117108404636383, -0.2989187240600586], [0.010385570116341114, 0.18124334514141083, 0.30283480882644653, -0.0135097

In [59]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

# Assume 'model' is your original model

# Get the output of the 4th last layer
input_to_last_3 = model.layers[-4].output

# Use this as the input to the last 3 layers
output_from_last_3 = input_to_last_3
for layer in model.layers[-3:]:  # Iterate through the last 3 layers
    output_from_last_3 = layer(output_from_last_3)

# Create a new model
new_model = Model(inputs=input_to_last_3, outputs=output_from_last_3)

# Save the new model
outputfinal = new_model.predict(output[2])
new_model.save('model_last_3_layers.h5')
new_model.summary()


1/1 [==============================] - 0s 35ms/step
Model: "model_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_25 (InputLayer)       [(None, 6, 6, 8)]         0         
                                                                 
 average_pooling2d_1 (Averag  (None, 3, 3, 8)          0         
 ePooling2D)                                                     
                                                                 
 flatten_1 (Flatten)         (None, 72)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                730       
                                                                 
Total params: 730
Trainable params: 730
Non-trainable params: 0
_________________________________________________________________
